<a href="https://colab.research.google.com/github/SimonADDA/ML-Poker-Hand/blob/main/ProjetML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
# reading csv files
df =  pd.read_csv('poker.data',sep=',',header=None)
print(df)

In [ ]:
df.head()

In [ ]:
df=df.rename(columns={0: "S1",1: "C1",2: "S2",3: "C2",4: "S3",5: "C3",6: "S4",7: "C4",8: "S5",9: "C5",10: "target"})
df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

print(df['target'].describe())
plt.figure(figsize=(9, 8))
sns.distplot(df['target'], color='g');

In [ ]:
import pandas as pd
import numpy as np

# generate binary values using get_dummies
df = pd.get_dummies(df, columns=['S1','S2','S3','S4','S5'])

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df_num = df.select_dtypes(include = ['int64'])
df_num.head()

In [ ]:
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8); # ; avoid having the matplotlib verbose informations

In [ ]:
for i in range(0, len(df_num.columns), 5):
    sns.pairplot(data=df_num,
                x_vars=df_num.columns[i:i+5],
                y_vars=['target'])

In [ ]:
corr = df_num.drop('target', axis=1).corr() # We already examined SalePrice correlations
plt.figure(figsize=(12, 10))

sns.heatmap(corr[(corr >= 0.5) | (corr <= -0.4)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

In [ ]:
# stratified sampling
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df["target"]):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]
    
for set_ in (strat_train_set, strat_test_set):
    set_.drop('target', axis=1, inplace=True)

In [ ]:
"""
================================
Nearest Neighbors Classification
================================

Sample usage of Nearest Neighbors classification.
It will plot the decision boundaries for each class.
"""
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets

n_neighbors = 3

# import the iris data to play with, keep only two input variables in order to make the plots at the end of the script
X = df.loc[:, df.columns != 'target']  # we only take the first two features. We could
                      # avoid this ugly slicing by using a two-dim dataset
y = df.target

In [ ]:
X

In [ ]:
y.shape

In [ ]:
clf = neighbors.KNeighborsClassifier(n_neighbors) # je précise la méthode et ses hyperparamètres
clf.fit(X, y) # je lance l'apprentissage

In [ ]:
# prevision
clf.predict(X)

In [ ]:
clf.score(X,y) # predit + calcul le score = la précision

In [ ]:
import random
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3,random_state=random.seed())
print(X_train.shape)
print(X_test.shape)

In [ ]:
n_neighbors=3
clf = neighbors.KNeighborsClassifier(n_neighbors) # je précise la méthode et ses hyperparamètres
clf.fit(X_train, Y_train) # je lance l'apprentissage
prev_test = clf.predict(X_test)
sc_train = clf.score(X_train,Y_train)
sc_test = clf.score(X_test,Y_test)
print(sc_train)
print(sc_test)

In [ ]:
# Compute the confusion matrix between kmean label and iris types
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(Y_test,prev_test)
# the confusion matrix is difficult to read as labels assigned by kmeans are arbitrary
print(cm)

In [ ]:
# Pour voir l'effet du partage ...

from sklearn.model_selection import KFold
kf=KFold(n_splits=4, shuffle=True) # partages de validation
XXX = X_train[:1000,:]  # je prends uniquement les 12 premières lignes
#print(XXX)
for learn,test in kf.split(XXX): # boucle sur différents partages de validation
  print("Learn")
  print(learn)
  print("test")
  print(test)

In [ ]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=3, shuffle=True) # partages de validation


from sklearn import neighbors
scores=[]
for k in range(1,10):  # les différentes valeurs de k à tester
    score=0
    clf=neighbors.KNeighborsClassifier(k)
    for learn,test in kf.split(X_train): # boucle sur différents partages de validation
        X_app=X_train[learn]
        Y_app=Y_train[learn]
        clf.fit(X_app,Y_app)
        X_val=X_train[test]
        Y_val=Y_train[test]
        score+=clf.score(X_val,Y_val)
    scores.append(score)
print(scores)
#plt(scores)
k_opt=scores.index(max(scores)) + 1  # valeur optimale de k
print(k_opt)

In [ ]:
h = .02  # step size in the mesh for the graphics

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

In [ ]:
# try two different values for the weights argument in knn

for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("3-Class classification (k = %i, weights = '%s')"
              % (n_neighbors, weights))

plt.show()

### Decision tree

In [ ]:
import numpy as np
from sklearn.datasets import load_digits

# https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits
digits=load_digits()
# print(dir(digits))
# print(digits.DESCR)
X=digits.data
Y=digits.target
print(digits.data.shape)
print(Y)